In [3]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
              
device  = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                        (0.5,0.5,0.5))
])

In [7]:
trainset = torchvision.datasets.CIFAR10(
    root= "./data", train=True,
    download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False,
    download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(
    trainset,batch_size=64, shuffle= True)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3,32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32,64, kernel_size=3,padding=1)
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(64*8*8,128)
        self.fc2 = nn.Linear(128,10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [13]:
epochs =10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):4f}")
    

Epoch 1, Loss: 1.351223
Epoch 2, Loss: 0.980841
Epoch 3, Loss: 0.824327
Epoch 4, Loss: 0.715238
Epoch 5, Loss: 0.618488
Epoch 6, Loss: 0.532104
Epoch 7, Loss: 0.448325
Epoch 8, Loss: 0.374245
Epoch 9, Loss: 0.306402
Epoch 10, Loss: 0.246003


In [15]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device),labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Test Accuracy: ",correct / total)

Test Accuracy:  0.7075
